## Question 1 

In [2]:
class NumericalIntegration:
    def __init__(self, f, a, b, n):
        self.f = f
        self.a = a
        self.b = b
        self.n = n

    def simpsons_rule(self):
        if self.n % 2 == 1:
            raise ValueError("n must be even for Simpson's rule.")
        
        h = (self.b - self.a) / self.n
        x = self.a
        result = self.f(x) + self.f(self.b)
        
        for i in range(1, self.n):
            x += h
            if i % 2 == 0:
                result += 2 * self.f(x)
            else:
                result += 4 * self.f(x)
        
        result *= h / 3
        return result

    def trapezoidal_rule(self):
        h = (self.b - self.a) / self.n
        result = (self.f(self.a) + self.f(self.b)) / 2.0
        
        for i in range(1, self.n):
            result += self.f(self.a + i * h)
        
        result *= h
        return result

    def midpoint_rule(self):
        h = (self.b - self.a) / self.n
        result = 0
        
        for i in range(self.n):
            result += self.f(self.a + (i + 0.5) * h)
        
        result *= h
        return result

    def gaussian_quadrature(self):
        if self.n == 2:
            # 2-point Gaussian quadrature
            nodes = [-0.5773502692, 0.5773502692]
            weights = [1.0, 1.0]
        elif self.n == 3:
            # 3-point Gaussian quadrature
            nodes = [-0.7745966692, 0.0, 0.7745966692]
            weights = [0.5555555556, 0.8888888889, 0.5555555556]
        elif self.n == 4:
            # 4-point Gaussian quadrature
            nodes = [-0.8611363116, -0.3399810435, 0.3399810435, 0.8611363116]
            weights = [0.3478548451, 0.6521451549, 0.6521451549, 0.3478548451]
        elif self.n == 6:
            # 6-point Gaussian quadrature
            nodes = [-0.9602898565, -0.7745966692, -0.4342437490, 0.4342437490, 0.7745966692, 0.9602898565]
            weights = [0.0471753364, 0.2920426860, 0.5688888889, 0.5688888889, 0.2920426860, 0.0471753364]
        else:
            raise ValueError("This implementation only supports 2, 3, 4, or 6-point Gaussian quadrature.")

        result = 0.0
        for i in range(self.n):
            # Convert nodes from [-1, 1] to [a, b]
            xi = ((self.b - self.a) / 2) * nodes[i] + (self.a + self.b) / 2
            result += weights[i] * self.f(xi)

        result *= (self.b - self.a) / 2
        return result


def q(x):
    return 1 - x - 4*x**3 +2*x**5

def RelError(Computed : float, true : float) -> float:
    return abs((Computed - true))*100/true

a = -2
b = 4
n = 6  # For Simpson's, Trapezoidal, and Mid-point (must be even for Simpson's)
TrueVal = 1104



integration = NumericalIntegration(q, a, b, n)
print(f'True Value : {TrueVal}')

print(f"Mid-point Rule Result: {integration.midpoint_rule()} | Relative Error(%) : {RelError(integration.midpoint_rule(),TrueVal)} %")

# Gaussian Quadrature with 4 points
integration_gauss_4 = NumericalIntegration(q, a, b, n=4)
print(f"4-point Gaussian Quadrature Result: {integration_gauss_4.gaussian_quadrature()} | Relative Error(%) : {RelError(integration_gauss_4.gaussian_quadrature(),TrueVal)} % ")

# Gaussian Quadrature with 6 points
integration_gauss_6 = NumericalIntegration(q, a, b, n=6)
print(f"6-point Gaussian Quadrature Result: {integration_gauss_6.gaussian_quadrature()} | Relative Error(%) : {RelError(integration_gauss_6.gaussian_quadrature(),TrueVal)} %")




True Value : 1104
Mid-point Rule Result: 1011.75 | Relative Error(%) : 8.355978260869565 %
4-point Gaussian Quadrature Result: 1103.9999998608978 | Relative Error(%) : 1.2599838081096886e-08 % 
6-point Gaussian Quadrature Result: 934.170626680158 | Relative Error(%) : 15.383095409405984 %


## Question 2 

In [3]:
import pandas as pd 
import numpy as np

# Creating a numpy array for the dataTable 
dataset = np.array([[336,294.4,266.4,260.8,260.5,249.6,193.6,165.6],[0.5,2,3,4,6,8,10,11]])
dataset = dataset.T
df = pd.DataFrame(columns= ['Pressure','Vol'] ,data=dataset) 

df

,Pressure,Vol
0,336.0,0.5
1,294.4,2.0
2,266.4,3.0
3,260.8,4.0
4,260.5,6.0
5,249.6,8.0
6,193.6,10.0
7,165.6,11.0


### Part a :

In [4]:
def Trapezoidal(data : pd.DataFrame , n=8 ) -> float:
    area = 0
    Vol = list(data['Vol'])
    P = list(data['Pressure'])
    for i in range(1,n):
        area += (Vol[i] - Vol[i-1])*(P[i]+P[i-1])*0.5
    return area


print(f'Trapezoidal Rule : {Trapezoidal(df)} kJ')

Trapezoidal Rule : 2670.9999999999995 kJ


### Part b:

In [5]:
def simpsons_rule(df : pd.DataFrame) -> float:
    
    x = df['Vol'].values
    y = df['Pressure'].values
    h = x[1] - x[0]
    n = len(x)

    integral = (h/3) * (y[0] + y[-1] + 4*np.sum(y[1:-1:2]) + 2*np.sum(y[2:-1:2]))
    return integral

# Example usage:
print(f' Pure Simpson\'s Rule : {simpsons_rule(df)} KJ')

 Pure Simpson's Rule : 2580.9 KJ


In [6]:
def adaptive_integration(df: pd.DataFrame) -> float:
    x = df['Vol'].values
    y = df['Pressure'].values
    h = x[1:] - x[:-1]  # calculate the differences between intervals

    integral = 0
    i = 0
    while i < len(x) - 1:
        # check if the next 3 sub-intervals are equal
        if i + 3 <= len(x) - 1 and np.allclose(h[i:i+3], h[i]):
            # apply Simpson's rule
            #print('1')
            integral += (h[i]/3) * (y[i] + y[i+3] + 4*y[i+1] + 2*y[i+2])
            i += 3
        else:
            # check if the next sub-interval is equal to the current one
            if i + 2 <= len(x) - 1 and np.allclose(h[i:i+2], h[i]):
                # apply 3/8 rule
                #print('2')
                integral += (3/8) * h[i] * (y[i] + y[i+2] + 3*y[i+1])
                i += 2
            else:
                # apply Trapezoidal rule
                #print('3')
                integral += (h[i] / 2) * (y[i] + y[i+1])
                i += 1

    return integral

print(f'Combined Approch Simpson\'s Rule {adaptive_integration(df)} KJ')

Combined Approch Simpson's Rule 2490.7000000000003 KJ
